# PyTorch Techniques for Model Optimization

## Table of Contents

[PyTorch Techniques for Model Optimization](#pytorch-techniques-for-model-optimization)

- [Saving Progress w/ Model Checkpointing](##saving-progress-w-model-checkpointing)

- [Model Training w/ Mini-Batches in PyTorch](##model-training-w-mini-batches-in-pytorch)

## Saving Progress w/ Model Checkpointing

**Model Checkpointing Intro.** - Will now focus on model checkpointing using PyTorch. This is vital technique in machine learning that allows save state of model during training, ensuring best-performing models are preserved. Will come to understand how implement model checkpointing, allowing to save model whenever achieves best performance on a validation set.

So model checkpointing involves saving state of a neural network model at various points during training process. Crucial for several reasons:
- **Prevent Loss of Progress**: In case of unexpected interruptions (e.g., power failure, hardware consumption), checkpointing helps resuming training from last saved state.
- **Save Best Performing Models**: By saving model whenever achieves a new best performance on validation set, ensure that retain best version of our model.

**Setting up Environment** - Assume set up environment seen before and used below: import necessary libraries, do preprocessing of Wine dataset, define model, define loss and optimizer. Will for now omit training loop with eval, graphing of loss and finally saving model loading and confirming same val_loss, as this code will be modified with checkpointing.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import sklearn.utils as skUtils

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from typing import List, Tuple, Dict

wine_set: skUtils.Bunch = load_wine()

def load_preprocessed_data(wine: skUtils.Bunch) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    X: np.ndarray = wine.data
    y: np.ndarray = wine.target

    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, stratify=y)

    scaler: StandardScaler = StandardScaler().fit(Xtrain)
    Xtrain_scaled: np.ndarray = scaler.transform(Xtrain)
    Xtest_scaled: np.ndarray = scaler.transform(Xtest)

    Xtrain_tensor: torch.Tensor = torch.tensor(Xtrain_scaled, dtype=torch.float32)
    Xtest_tensor: torch.Tensor = torch.tensor(Xtest_scaled, dtype=torch.float32)
    ytrain_tensor: torch.Tensor = torch.tensor(ytrain, dtype=torch.long)
    ytest_tensor: torch.Tensor = torch.tensor(ytest, dtype=torch.long)

    return Xtrain_tensor, Xtest_tensor, ytrain_tensor, ytest_tensor


X_train, X_test, y_train, y_test = load_preprocessed_data(wine_set)

model: nn.Sequential = nn.Sequential( # MAKE SURE YOU CAN CUSTOM DEFINE TOO!!
    nn.Linear(in_features=13, out_features=10),
    nn.ReLU(),
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 3)
)

criterion: nn.CrossEntropyLoss = nn.CrossEntropyLoss()
optimizer: optim.Adam = optim.Adam(model.parameters(), lr=0.001)

**Initialize Checkpoint Parameters** - Before diving into training loops changes, first set up initial parameters for checkpointing. Will ensure can effectively track model's performance and save best version. Specifically need establish:
- `best_loss` to keep track of best validation loss. Initialize `best_loss` to `float(inf)` to ensure first validation loss will trigger model save.
- `checkpoint_path` where model will be saved.

In [2]:
best_loss: float = float('inf')
checkpoint_path: str = "best_model.pth"

**Training Loop with Checkpointing** - Now implement training loop portion with validation and model checkpointing.

In [3]:
num_epochs: int = 150
history: Dict[str, List[float]] = {"loss": [], "val_loss": []}
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    history["loss"].append(loss.item())

    model.eval()
    with torch.no_grad():
        outputs_test = model(X_test)
        val_loss: float = criterion(outputs_test, y_test).item()
        history["val_loss"].append(val_loss)
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model, checkpoint_path)
        print(f'Model saved at epoch {epoch} with validation loss {val_loss:.4f}')

    if epoch % 10 == 0:
        print(f'At epoch {epoch}/{num_epochs}, Loss is: {loss.item():.4f} and Val. Loss is: {val_loss:.4f}')

Model saved at epoch 0 with validation loss 1.1336
At epoch 0/150, Loss is: 1.1362 and Val. Loss is: 1.1336
Model saved at epoch 1 with validation loss 1.1296
Model saved at epoch 2 with validation loss 1.1257
Model saved at epoch 3 with validation loss 1.1219
Model saved at epoch 4 with validation loss 1.1181
Model saved at epoch 5 with validation loss 1.1143
Model saved at epoch 6 with validation loss 1.1107
Model saved at epoch 7 with validation loss 1.1071
Model saved at epoch 8 with validation loss 1.1035
Model saved at epoch 9 with validation loss 1.1000
Model saved at epoch 10 with validation loss 1.0965
At epoch 10/150, Loss is: 1.0989 and Val. Loss is: 1.0965
Model saved at epoch 11 with validation loss 1.0931
Model saved at epoch 12 with validation loss 1.0897
Model saved at epoch 13 with validation loss 1.0862
Model saved at epoch 14 with validation loss 1.0829
Model saved at epoch 15 with validation loss 1.0796
Model saved at epoch 16 with validation loss 1.0763
Model saved

In this training loop:
- Model is trained on training set
- Model's performance is validated on the validation set
- If validation loss improves, the model is saved using `torch.save()` . This ensures that only best performing model is saved.

Learned concept and importance of model checkpointing, as well as how implement checkpointing in a PyTorch model. Remember that implementing effective checkpointing can significantly boost productivity and model performance in real-world machine learning tasks.

## Model Training w/ Mini-Batches in PyTorch

Here learn how to efficiently train neural network model using mini-batches in PyTorch. Focus will be on understanding concept of mini-batches, creating them using PyTorch's `DataLoader` and training model using these mini-batches. Will be equipped with knowledge to implement mini-batch gradient descent in machine learning projects.

**Intro to Mini-Batch Training** - In machine learning there are three main methods for training models: stochastic gradient descent (SGD), full-batch gradient descent, and mini-batch gradient descent. Explained here using simple analogy.

Imagine learning to shoot basketballs in hoop:
1. **Stochastic Gradient Descent (SGD)**: This is like shooting one basketball, adjusting your aim after each shot. Get feedback quickly, but each shot influenced by random factors, making learning process noisy.
2. **Full-Batch Gradient Descent**: This is like shooting all basketballs you have, then reviewing overall performance to adjust your aim. Gives clear picture but is slow and tiring because have to shoot all balls before making any adjustments. 
3. **Mini-Batch Gradient Descent**: This method is middle ground. Like shooting few basketballs (say 10) before adjusting your aim. Faster than shooting all balls at once and more stable than adjusting after every single shot, offering balanced approach.

**Why Use Mini-Batch Training?**
- 1. **Efficiency**: Processing smaller subsets of data significantly reduces memory usage and can take advatntaged of parllel processing hardware.
- 2. **Convergence**: Provides balance between noisy updates (SGD) and slow updates (full-batch), which can stabilize convergence.
- 3. **Regularization**: Each mini-batch introduces some noise into parameter updates, which can help overfitting.

**Loading the Dataset** -
After having loaded dataset preprocessed and returned as PyTorch tensors, can use `DataLoader` to divide dataset into mini-batches and iterate over them efficiently.

In [4]:
from torch.utils.data import TensorDataset, DataLoader
batch_size: int = 32
dataset: TensorDataset = TensorDataset(X_train, y_train)
dataloader: DataLoader = DataLoader(dataset, batch_size, shuffle=True)

In code above:
- `TensorDataset`: Combines features `X` and targets `y` into single dataset.
- `DataLoader`: Splits dataset into mini-batches of size specified by `batch_size` , making it easy to iterate over dataset in chunks during training.

By setting `batch_size=32` each mini-batch will contain 32 samples. The `shuffle=True` parameter ensures data shuffled at each epoch, improving the generalization capabilities of model. The `DataLoader` simplifies the process of batching and shuffling, which essential for efficient mini-batch training.

**Building and Compiling Model** - Before using dataset split into mini-batches, the standard process of the model definition, loss function and optimizer are needed. These are copied from prev. section and doing `del` before to ensure data is reset.

In [5]:
del model, criterion, optimizer

model: nn.Sequential = nn.Sequential(
    nn.Linear(in_features=13, out_features=10),
    nn.ReLU(),
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 3)
)

criterion: nn.CrossEntropyLoss = nn.CrossEntropyLoss()
optimizer: optim.Adam = optim.Adam(model.parameters(), lr=0.001)

**Training Model with Mini-Batches** - Now dataset, model, loss function and optimizer are ready, can begin training model using mini-batches. Here is training implementation:

In [6]:
num_epochs: int = 10
for epoch in range(num_epochs):
    model.train()
    running_loss: float = 0.0
    for batch in dataloader:
        optimizer.zero_grad()
        batch_X: torch.Tensor = batch[0]
        batch_y: torch.Tensor = batch[1]
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        print(f'Batch Loss: {loss.item():.4f}')
        running_loss += loss.item() * batch_X.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}] Average Loss: {epoch_loss:.4f}')

Batch Loss: 1.1119
Batch Loss: 1.1269
Batch Loss: 1.1133
Batch Loss: 1.1033
Epoch [1/10] Average Loss: 1.1142
Batch Loss: 1.1070
Batch Loss: 1.1220
Batch Loss: 1.1083
Batch Loss: 1.0652
Epoch [2/10] Average Loss: 1.1018
Batch Loss: 1.0913
Batch Loss: 1.0950
Batch Loss: 1.0861
Batch Loss: 1.0860
Epoch [3/10] Average Loss: 1.0897
Batch Loss: 1.1016
Batch Loss: 1.0786
Batch Loss: 1.0617
Batch Loss: 1.0681
Epoch [4/10] Average Loss: 1.0778
Batch Loss: 1.0724
Batch Loss: 1.0769
Batch Loss: 1.0593
Batch Loss: 1.0546
Epoch [5/10] Average Loss: 1.0662
Batch Loss: 1.0365
Batch Loss: 1.0523
Batch Loss: 1.0734
Batch Loss: 1.0571
Epoch [6/10] Average Loss: 1.0548
Batch Loss: 1.0643
Batch Loss: 1.0191
Batch Loss: 1.0368
Batch Loss: 1.0560
Epoch [7/10] Average Loss: 1.0437
Batch Loss: 1.0290
Batch Loss: 1.0444
Batch Loss: 1.0182
Batch Loss: 1.0387
Epoch [8/10] Average Loss: 1.0324
Batch Loss: 1.0151
Batch Loss: 1.0109
Batch Loss: 1.0174
Batch Loss: 1.0429
Epoch [9/10] Average Loss: 1.0209
Batch Loss

Now break down how works with mini-batches:

1. **Iterating through Batches**: The loop `for batch_X, batch_y in dataloader` lets model process dataset in smaller chunks called mini-batches. Then `batch_X` contains input features, and `batch_y` contains corresponding labels for each mini-batch.

2. **Calculating Batch Loss**: For each mini-batch, model makes predictions (`outputs`), and loss(`loss`) is computed by comparing these predictions to actual labels. The loss for each mini-batch is printed using `loss.item()`, providing immediate feedback.

3. **Accumulating Loss**: Variable `running_loss` keeps cumulative total of loss for entire epoch. For each mini-batch, you add product of batch loss (`loss.item()`) and number of samples in that batch (`batch_X.size(0)`) to `running_loss` . This scaling is necessary because mini-batches can have different sizes, especially last mini-batch which might be smaller. By multiplying loss by the batch size, ensure each sample in dataset contributes equally to total loss, making final epoch loss calculation accurate.

4. **Computing Epoch Loss**: At end of each epoch, average loss (`epoch_loss`) calculated by dividing `running_loss` by total number of samples in dataset(`len(dataloader.dataset)`). This gives normalized measure of loss over entire dataset, offering clear indication of model's performance for each epoch.

Working with mini-batches helps make training process more efficient and provides way to handle larger datasets that wouldn't fit into memory all at once.